# Imports

In [1]:
import numpy as np

# Add these lines if you clone from https://github.com/fdalvi/NeuroX
# import sys
# sys.path.append("/path/to/NeuroX")
# In the case of the notebook, the path is the parent directory
import sys
sys.path.append("..")

# Data

In [2]:
!cat test.in

This is a test sentence .
This is another sentence .
The NeuroX toolkit can be used for neuron analysis !
This sentence contains an <<UNKNOWN>> word .
Individual neuron activations can be insightful .
It is a beautiful day outside .
The red car zoomed by quickly !
Is this a question ?
There are more than 10000 neurons in some models .
Hope you are enjoying the tutorial !

In [3]:
!cat test.label

DT VBZ DT NN NN .
DT VBZ DT NN .
DT NNP NN MD VB VBN IN JJ NN .
DT NN VBZ DT NN NN .
NNP CC NNS MD VB JJ .
PRP VBZ DT JJ NN IN .
DT JJ NN VBN IN RB .
VBZ DT DT NN .
EX VBP JJR IN CD NNS IN DT NNS .
NN PRP VBP VBG DT NN .

# Extract Representations

In [4]:
import neurox.data.extraction.transformers_extractor as transformers_extractor
transformers_extractor.extract_representations('bert-base-uncased',
    'test.in',
    'activations.json',
    aggregation="average" #last, first
)

Loading model: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Reading input corpus
Preparing output file
Extracting representations from model
Sentence         : "This is a test sentence ."
Original    (006): ['This', 'is', 'a', 'test', 'sentence', '.']
Tokenized   (008): ['[CLS]', 'this', 'is', 'a', 'test', 'sentence', '.', '[SEP]']
Filtered   (006): ['this', 'is', 'a', 'test', 'sentence', '.']
Detokenized (006): ['this', 'is', 'a', 'test', 'sentence', '.']
Counter: 6
Hidden states:  (13, 6, 768)
# Extracted words:  6
Sentence         : "This is another sentence ."
Original    (005): ['This', 'is', 'another', 'sentence', '.']
Tokenized   (007): ['[CLS]', 'this', 'is', 'another', 'sentence', '.', '[SEP]']
Filtered   (005): ['this', 'is', 'another', 'sentence', '.']
Detokenized (005): ['this', 'is', 'another', 'sentence', '.']
Counter: 5
Hidden states:  (13, 5, 768)
# Extracted words:  5
Sentence         : "The NeuroX toolkit can be used for neuron analysis !"
Original    (010): ['The', 'NeuroX', 'toolkit', 'can', 'be', 'used', 'for', 'neuron', 'a

In [5]:
!cut -c1-120 < activations.json

{"linex_index": 0, "features": [{"token": "this", "layers": [{"index": 0, "values": [-0.6485135, 0.67392403, -0.09324985
{"linex_index": 1, "features": [{"token": "this", "layers": [{"index": 0, "values": [-0.6485135, 0.67392403, -0.09324985
{"linex_index": 2, "features": [{"token": "the", "layers": [{"index": 0, "values": [-0.43669653, 0.53601718, -0.05141334
{"linex_index": 3, "features": [{"token": "this", "layers": [{"index": 0, "values": [-0.6485135, 0.67392403, -0.09324985
{"linex_index": 4, "features": [{"token": "individual", "layers": [{"index": 0, "values": [0.14265811, 0.75676745, -0.91
{"linex_index": 5, "features": [{"token": "it", "layers": [{"index": 0, "values": [-0.42272022, -0.02889436, -0.14556105
{"linex_index": 6, "features": [{"token": "the", "layers": [{"index": 0, "values": [-0.43669653, 0.53601718, -0.05141334
{"linex_index": 7, "features": [{"token": "is", "layers": [{"index": 0, "values": [-0.29041198, 0.00098542, -0.43960977,
{"linex_index": 8, "features": [

# Prepare Data

In [6]:
import neurox.data.loader as data_loader
activations, num_layers = data_loader.load_activations('activations.json', 768)

Loading json activations from activations.json...
10 13.0


In [7]:
print(activations)

[array([[-0.6485135 ,  0.67392403, -0.09324985, ..., -0.5617491 ,
         0.48687834,  0.54459047],
       [-0.62703037, -0.06331295, -0.31427881, ..., -0.15095142,
         0.20867236,  0.97557974],
       [ 0.39637804,  0.31565616,  0.02008923, ..., -0.07199516,
         0.2717956 ,  1.4120574 ],
       [ 0.60103226, -0.69701707, -0.2000732 , ..., -0.72475618,
         0.08129568, -0.25088236],
       [ 0.83230692,  0.28782725,  0.00214796, ..., -0.11331076,
        -0.15800183, -0.34240773],
       [-0.078184  ,  0.01739771, -0.28644535, ...,  0.20194323,
        -0.62845349, -0.45307434]]), array([[-0.6485135 ,  0.67392403, -0.09324985, ..., -0.1489853 ,
         0.25174388,  0.49430278],
       [-0.62703037, -0.06331295, -0.31427881, ..., -0.08159983,
         0.22553164,  1.23856652],
       [ 0.17470703,  0.5296641 ,  0.15547034, ..., -0.02144399,
         0.16490568,  0.45266795],
       [ 0.79593539,  0.43478003,  0.19534861, ...,  0.24064474,
        -0.1610865 , -0.04338051

In [8]:
activations[0].shape # 9984 -> 768 * 13

(6, 9984)

In [9]:
activations[1].shape

(5, 9984)

In [10]:
# load_data also does sanity checks for parallelism between tokens, labels and activations
tokens = data_loader.load_data('test.in',
                               'test.label',
                               activations,
                               512 # max_sent_l
                              )

In [11]:
tokens

{'source': [['This', 'is', 'a', 'test', 'sentence', '.'],
  ['This', 'is', 'another', 'sentence', '.'],
  ['The',
   'NeuroX',
   'toolkit',
   'can',
   'be',
   'used',
   'for',
   'neuron',
   'analysis',
   '!'],
  ['This', 'sentence', 'contains', 'an', '<<UNKNOWN>>', 'word', '.'],
  ['Individual', 'neuron', 'activations', 'can', 'be', 'insightful', '.'],
  ['It', 'is', 'a', 'beautiful', 'day', 'outside', '.'],
  ['The', 'red', 'car', 'zoomed', 'by', 'quickly', '!'],
  ['Is', 'this', 'a', 'question', '?'],
  ['There',
   'are',
   'more',
   'than',
   '10000',
   'neurons',
   'in',
   'some',
   'models',
   '.'],
  ['Hope', 'you', 'are', 'enjoying', 'the', 'tutorial', '!']],
 'target': [['DT', 'VBZ', 'DT', 'NN', 'NN', '.'],
  ['DT', 'VBZ', 'DT', 'NN', '.'],
  ['DT', 'NNP', 'NN', 'MD', 'VB', 'VBN', 'IN', 'JJ', 'NN', '.'],
  ['DT', 'NN', 'VBZ', 'DT', 'NN', 'NN', '.'],
  ['NNP', 'CC', 'NNS', 'MD', 'VB', 'JJ', '.'],
  ['PRP', 'VBZ', 'DT', 'JJ', 'NN', 'IN', '.'],
  ['DT', 'JJ', 'NN'

In [12]:
import neurox.interpretation.utils as utils
X, y, mapping = utils.create_tensors(tokens, activations, 'NN')
label2idx, idx2label, src2idx, idx2src = mapping

Number of tokens:  71
length of source dictionary:  51
length of target dictionary:  19
71
Total instances: 71
['Hope', 'some', 'question', 'enjoying', 'quickly', 'are', 'sentence', 'models', 'toolkit', 'used', 'more', 'outside', 'neuron', '?', '<<UNKNOWN>>', 'in', 'beautiful', 'day', 'car', 'a']


In [13]:
# X_dev, y_dev, mapping = utils.create_tensors(dev_tokens, dev_activations, 'LABEL1', mapping=mapping)
X[0] #9984

array([-0.6485135 ,  0.673924  , -0.09324985, ..., -0.5617491 ,
        0.48687834,  0.5445905 ], dtype=float32)

# Train Probing Classifier

In [14]:
import neurox.interpretation.linear_probe as linear_probe
probe = linear_probe.train_logistic_regression_probe(X, y, lambda_l1=0.001, lambda_l2=0.001)

Training classification probe
Creating model...
Number of training instances: 71
Number of classes: 19


epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.1469


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.0455


...

Epoch: [8/10], Loss: 0.0378


epoch [9/10]: 0it [00:00, ?it/s]

Epoch: [9/10], Loss: 0.0359


epoch [10/10]: 0it [00:00, ?it/s]

Epoch: [10/10], Loss: 0.0337


# Evaluate Model

In [15]:
linear_probe.evaluate_probe(probe, X, y, idx_to_class=idx2label)

Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 1.00


{'__OVERALL__': 1.0,
 'NNP': 1.0,
 'IN': 1.0,
 'PRP': 1.0,
 'NN': 1.0,
 'JJR': 1.0,
 '.': 1.0,
 'CC': 1.0,
 'VBZ': 1.0,
 'VB': 1.0,
 'JJ': 1.0,
 'RB': 1.0,
 'VBP': 1.0,
 'VBG': 1.0,
 'VBN': 1.0,
 'EX': 1.0,
 'DT': 1.0,
 'CD': 1.0,
 'NNS': 1.0,
 'MD': 1.0}

# Layerwise probing

In [16]:
# [layer0_neuron0, layer0_neuron1, layer0_neuron2 ...., layer1_neuron0, layer1_neuron1,....., layer13_neuron768]
# [768,768,770...1536] <- for layer 1
import neurox.interpretation.ablation as ablation

In [17]:
layer_0_X = ablation.filter_activations_by_layers(X, [0], 13)
print(layer_0_X.shape)
probe_layer_0 = linear_probe.train_logistic_regression_probe(layer_0_X, y, lambda_l1=0.001, lambda_l2=0.001)
linear_probe.evaluate_probe(probe_layer_0, layer_0_X, y, idx_to_class=idx2label)

(71, 768)
Training classification probe
Creating model...
Number of training instances: 71
Number of classes: 19


epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.1378


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.1137


...

epoch [10/10]: 0it [00:00, ?it/s]

Epoch: [10/10], Loss: 0.0295


Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 0.99


{'__OVERALL__': 0.9859154929577465,
 'NNP': 1.0,
 'IN': 1.0,
 'PRP': 1.0,
 'NN': 1.0,
 'JJR': 1.0,
 '.': 1.0,
 'CC': 1.0,
 'VBZ': 1.0,
 'VB': 1.0,
 'JJ': 0.75,
 'RB': 1.0,
 'VBP': 1.0,
 'VBG': 1.0,
 'VBN': 1.0,
 'EX': 1.0,
 'DT': 1.0,
 'CD': 1.0,
 'NNS': 1.0,
 'MD': 1.0}

# Get Neuron Ranking

In [18]:
ordering, cutoffs = linear_probe.get_neuron_ordering(probe, label2idx)

  0%|          | 0/101 [00:00<?, ?it/s]

In [19]:
ordering[:10]

[6273, 4483, 5382, 8840, 8264, 2444, 8150, 1494, 4698, 6109]

# Train on top N neurons

In [20]:
X_selected = ablation.filter_activations_keep_neurons(X, ordering[:100])

In [21]:
X_selected.shape

(71, 100)

In [22]:
probe_selected = linear_probe.train_logistic_regression_probe(X_selected, y, lambda_l1=0.001, lambda_l2=0.001)

Training classification probe
Creating model...
Number of training instances: 71
Number of classes: 19


epoch [1/10]: 0it [00:00, ?it/s]

Epoch: [1/10], Loss: 0.1349


epoch [2/10]: 0it [00:00, ?it/s]

Epoch: [2/10], Loss: 0.1302


...

Epoch: [8/10], Loss: 0.1062


epoch [9/10]: 0it [00:00, ?it/s]

Epoch: [9/10], Loss: 0.1026


epoch [10/10]: 0it [00:00, ?it/s]

Epoch: [10/10], Loss: 0.0992


In [23]:
linear_probe.evaluate_probe(probe_selected, X_selected, y, idx_to_class=idx2label)

Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 0.44


{'__OVERALL__': 0.43661971830985913,
 'NNP': 0.0,
 'IN': 0.2,
 'PRP': 0.5,
 'NN': 0.38461538461538464,
 'JJR': 0.0,
 '.': 1.0,
 'CC': 0.0,
 'VBZ': 0.2,
 'VB': 0.0,
 'JJ': 0.25,
 'RB': 0.0,
 'VBP': 1.0,
 'VBG': 0.0,
 'VBN': 0.0,
 'EX': 0.0,
 'DT': 0.6153846153846154,
 'CD': 0.0,
 'NNS': 0.6666666666666666,
 'MD': 0.0}

# Further analysis

In [24]:
top_neurons, top_neurons_per_class = linear_probe.get_top_neurons(probe, 0.01, label2idx)
print(top_neurons)

[   5 2061 8206 6163 8213 6169 4121 4134 2094 4143   56 4156 4157 4163
 6214 8264 2124 8269 4174 2134 2136 2138 6236 4191 4192 6245 6246 4199
 2160 6262 4218  126 6273 2180  143 2198 2203 6302 6304 2213 6313  178
  181 8374  182  184 6332 6335  199 8396 6350 4306 2269 6366 4329 6379
 4333 2286 6381 4339  246 8449  258  260 6417 6418 8474 4389 2342 2344
 4393 2348 6445 8495 6452  308  313 8505  317 6467 8516 6472 4426  335
 4436 8537 6490 2394  349 8545 2410 6508 2412 2421 6519 2428 6526 4480
 4483 2441 2443 2444 4496 8594  416 4520 4522 4523 4526 4528 6584 2491
 2493 2494 6593  454  468 8662  497 6643 6646 2551 2559 8708 4623 4634
 4642  549 8742  554 8750 4656 6708  583 4679 2638 6743 4698 2657  624
 4721 4725 2677 4727 8825  637  644 8840 6792 2708 8853 2709 4760 2713
 4767 2721 6822 6823 4780 4784 4791 4796  706 4803 2754 2758 2764 6862
 2766 6868 6869  724 6872 6873  744 4841 2794 4842  751 2801 2805 8961
  770 6913 8965  778 4885 4889  793 4902  806 4909 4916  824 9022  839
  840 

In [25]:
X_zeroed = ablation.zero_out_activations_keep_neurons(X, top_neurons)
linear_probe.evaluate_probe(probe, X_zeroed, y, idx_to_class=idx2label)

Evaluating: 0it [00:00, ?it/s]

Score (accuracy) of the probe: 0.96


{'__OVERALL__': 0.9577464788732394,
 'NNP': 1.0,
 'IN': 1.0,
 'PRP': 1.0,
 'NN': 1.0,
 'JJR': 1.0,
 '.': 1.0,
 'CC': 1.0,
 'VBZ': 1.0,
 'VB': 1.0,
 'JJ': 1.0,
 'RB': 1.0,
 'VBP': 1.0,
 'VBG': 1.0,
 'VBN': 0.5,
 'EX': 1.0,
 'DT': 1.0,
 'CD': 1.0,
 'NNS': 1.0,
 'MD': 0.0}

In [26]:
import neurox.analysis.visualization as visualization

# Visualize a single sentence
visualization.visualize_activations(
    tokens["source"][9],
    activations[9][:, 2]
)

In [27]:
# Visualize entire dataset using transformer model
from IPython.display import display
from neurox.analysis.visualization import TransformersVisualizer

viz = TransformersVisualizer('bert-base-uncased')
for s_idx in range(len(tokens["source"])):
    display(viz(tokens["source"][s_idx], 0, 5))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
